pmgus_dev_stuff.ipynb

In [39]:
import pandas as pd
import numpy as np
import requests
import os
import json
import yfinance as yf
import datetime
from datetime import datetime, timedelta

In [40]:
import dotenv
from dotenv import load_dotenv
load_dotenv()

True

In [41]:
# my test tickers
ticker_list = ['SOFI','PLTR','NOW','ROIC']

get pre-market price data for SOFI
-> do i need a websocket for pre-market data? after hours, post market ours data?
get pre-market clone data of TradingView
add features new and other ideas that we find

# POLYGONIO

In [42]:
def get_snapshot(ticker, api_key):
    url = f"https://api.polygon.io/v2/snapshot/locale/us/markets/stocks/tickers/{ticker}?apiKey={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

In [43]:
#### PolygonIo DEVELOPEMENT ####
ticker = ticker_list[0]
api_key = os.getenv("polygon_api_key")  
snapshot_data = get_snapshot(ticker, api_key)

# Print the raw JSON response in a nice format
if snapshot_data:
    print(json.dumps(snapshot_data, indent=2))
    
    # If you want to extract specific fields, for example the last trade price:
    if 'ticker' in snapshot_data and 'lastTrade' in snapshot_data['ticker']:
        last_trade_price = snapshot_data['ticker']['lastTrade'].get('p')
        print(f"\nThe most recent trade price for {ticker} is: {last_trade_price}")
    else:
        print("\nNo 'lastTrade' data found in the snapshot response.")

{
  "ticker": {
    "ticker": "SOFI",
    "todaysChangePerc": 3.807740324594254,
    "todaysChange": 0.6099999999999994,
    "updated": 1735261200000000000,
    "day": {
      "o": 15.98,
      "h": 16.68,
      "l": 15.88,
      "c": 16.6,
      "v": 36892700.0,
      "vw": 16.4665
    },
    "lastQuote": {
      "P": 16.63,
      "S": 49,
      "p": 16.61,
      "s": 1,
      "t": 1735261199752001700
    },
    "lastTrade": {
      "c": [
        12
      ],
      "i": "8163",
      "p": 16.63,
      "s": 308,
      "t": 1735261194971047072,
      "x": 11
    },
    "min": {
      "av": 36892700.0,
      "t": 1735261140000,
      "n": 16,
      "o": 16.63,
      "h": 16.63,
      "l": 16.59,
      "c": 16.63,
      "v": 1188,
      "vw": 16.6162
    },
    "prevDay": {
      "o": 15.73,
      "h": 16.03,
      "l": 15.57,
      "c": 16.02,
      "v": 15729659.0,
      "vw": 15.8539
    }
  },
  "status": "OK",
  "request_id": "e0af1ac1683f1c6765d41b18275cd545"
}

The most recent trad

In [44]:
def get_enhanced_snapshot(ticker, api_key):
    base_url = "https://api.polygon.io/v2"
    snapshot_url = f"{base_url}/snapshot/locale/us/markets/stocks/tickers/{ticker}"
    # Get previous day's close and pre-market data
    aggs_url = f"{base_url}/aggs/ticker/{ticker}/range/1/minute/previous"
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    
    try:
        snapshot_response = requests.get(snapshot_url, headers=headers)
        snapshot_response.raise_for_status()
        snapshot_data = snapshot_response.json()
        
        # Calculate pre-market metrics
        current_price = snapshot_data['ticker']['lastTrade']['p']
        prev_close = snapshot_data['ticker']['prevDay']['c']
        
        # Calculate pre-market changes
        pm_change = current_price - prev_close
        pm_change_percent = (pm_change / prev_close) * 100
        
        # Enhance the snapshot data with pre-market metrics
        enhanced_data = {
            'Symbol': ticker,
            'Price': current_price,
            'prev_close': prev_close,
            'pm_change': pm_change,
            'Pre-market Change %': pm_change_percent,
            'volume': snapshot_data['ticker']['day']['v'],
            'vwap': snapshot_data['ticker']['day']['vw'],
            'timestamp': snapshot_data['ticker']['updated']
        }
        
        return enhanced_data
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {ticker}: {str(e)}")
        return None

In [45]:
def build_screening_df(tickers, api_key):
    """
    Build a DataFrame with screening data for multiple tickers
    """
    results = []
    
    for ticker in tickers:
        data = get_enhanced_snapshot(ticker, api_key)
        if data:
            results.append(data)
    
    df = pd.DataFrame(results)
    
    # Add additional calculated columns
    if not df.empty:
        df['rel_volume'] = df['volume'] / df['volume'].mean()
        # YOU CAN ADD STUFF HERE LATER        
    return df

# execute
polygon_df = build_screening_df(ticker_list, api_key)

In [46]:
display(polygon_df)

,Symbol,Price,prev_close,pm_change,Pre-market Change %,volume,vwap,timestamp,rel_volume
0,SOFI,16.63,16.02,0.61,3.807740,36892700.0,16.4665,1735261200000000000,1.505952
1,PLTR,81.82,82.38,-0.56,-0.679777,60321658.0,82.3025,1735261200000000000,2.462317
2,NOW,1099.00,1105.56,-6.56,-0.593364,425125.0,1099.4303,1735258920000000000,0.017354
3,ROIC,17.35,17.38,-0.03,-0.172612,352225.0,17.3647,1735246860000000000,0.014378


# Yahoo Finance

In [47]:
#### Yahoo DEVELOPEMENT ####
print(ticker_list[1])
# Create a Ticker object
ticker = yf.Ticker(ticker_list[1])
# Retrieve the info dictionary, which often includes floatShares, sharesOutstanding, etc.
info = ticker.info
# Print the ticker symbol
print(f"Ticker: {ticker_list[1]}")
# Print all the keys in the info dictionary (useful for discovering other available data)
print("\nTicker Info Keys:", info.keys())
# Print float shares and shares outstanding if they exist in the info dict
print("\nFloat Shares:", info.get('floatShares'))
print("Shares Outstanding:", info.get('sharesOutstanding'))


PLTR
Ticker: PLTR

Ticker Info Keys: dict_keys(['address1', 'address2', 'city', 'state', 'zip', 'country', 'phone', 'website', 'industry', 'industryKey', 'industryDisp', 'sector', 'sectorKey', 'sectorDisp', 'longBusinessSummary', 'fullTimeEmployees', 'companyOfficers', 'auditRisk', 'boardRisk', 'compensationRisk', 'shareHolderRightsRisk', 'overallRisk', 'governanceEpochDate', 'compensationAsOfEpochDate', 'maxAge', 'priceHint', 'previousClose', 'open', 'dayLow', 'dayHigh', 'regularMarketPreviousClose', 'regularMarketOpen', 'regularMarketDayLow', 'regularMarketDayHigh', 'beta', 'trailingPE', 'forwardPE', 'volume', 'regularMarketVolume', 'averageVolume', 'averageVolume10days', 'averageDailyVolume10Day', 'bid', 'ask', 'bidSize', 'askSize', 'marketCap', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'priceToSalesTrailing12Months', 'fiftyDayAverage', 'twoHundredDayAverage', 'currency', 'enterpriseValue', 'profitMargins', 'floatShares', 'sharesOutstanding', 'sharesShort', 'sharesShortPriorMonth', 'sh

In [48]:
import yfinance as yf
import pandas as pd

# List of tickers to fetch data for
print(ticker_list)
# This list will hold dictionaries of our extracted data
data_rows = []

for symbol in ticker_list:
    ticker_obj = yf.Ticker(symbol)
    info = ticker_obj.info
    
    # Build a row of data
    row = {
        "Symbol": symbol,  # Or you could use info.get("symbol") if you trust Yahoo's symbol field
        "FloatShares": info.get('floatShares'),
        "Float shares outstanding": info.get('sharesOutstanding'),
        # Add other fields here if needed
        'Industry': info.get('industry'),   
        'Sector': info.get('sector'),
        'Market capitalization': info.get('marketCap'),
        'averageDailyVolume10Day': info.get('averageDailyVolume10Day'),
        'sharesShort': info.get('sharesShort'),
    }
    data_rows.append(row)

# Convert the list of dictionaries into a pandas DataFrame
yahoo_df = pd.DataFrame(data_rows)
display(yahoo_df)

['SOFI', 'PLTR', 'NOW', 'ROIC']


,Symbol,FloatShares,Float shares outstanding,Industry,Sector,Market capitalization,averageDailyVolume10Day,sharesShort
0,SOFI,1065212727,1085149952,Credit Services,Financial Services,18013489152,47056670,136303467
1,PLTR,2097077792,2180649984,Software - Infrastructure,Technology,187116568576,109028050,76144399
2,NOW,205718681,206316992,Software - Application,Technology,226765062144,1248060,3213706
3,ROIC,124391879,127788000,REIT - Retail,Real Estate,2217121792,1386090,1478454


In [51]:
# Merge the two dataframes
pgyf_df = pd.merge(polygon_df, yahoo_df, on="Symbol", how="inner")
print(len(pgyf_df.columns))
display(pgyf_df)

16


,Symbol,Price,prev_close,pm_change,Pre-market Change %,volume,vwap,timestamp,rel_volume,FloatShares,Float shares outstanding,Industry,Sector,Market capitalization,averageDailyVolume10Day,sharesShort
0,SOFI,16.63,16.02,0.61,3.807740,36892700.0,16.4665,1735261200000000000,1.505952,1065212727,1085149952,Credit Services,Financial Services,18013489152,47056670,136303467
1,PLTR,81.82,82.38,-0.56,-0.679777,60321658.0,82.3025,1735261200000000000,2.462317,2097077792,2180649984,Software - Infrastructure,Technology,187116568576,109028050,76144399
2,NOW,1099.00,1105.56,-6.56,-0.593364,425125.0,1099.4303,1735258920000000000,0.017354,205718681,206316992,Software - Application,Technology,226765062144,1248060,3213706
3,ROIC,17.35,17.38,-0.03,-0.172612,352225.0,17.3647,1735246860000000000,0.014378,124391879,127788000,REIT - Retail,Real Estate,2217121792,1386090,1478454
